## Lecture 16: Build Neural Networks Using Keras

### Keras
- "Keras is a software library that provides Python APIs for artificial neural networks. Keras acts as an interface for the TensorFlow library"
- Learn Keras: https://keras.io/about/
- Install Keras
        We will need to have the TensorFlow package installed to use Kearas. https://www.tensorflow.org/install
        
        

In [1]:
from tensorflow import keras
keras.__version__

'2.8.0'

### Task: Breast cancer classification using NNs
- Prepare the dataset
    - load the dataset
    - prepare the training and test sets
- Build a NN using Keras
- Train the NN model
- Evaluate the NN model

#### Load the dataset

In [2]:
import sklearn.datasets as ds
breast_ds = ds.load_breast_cancer()
print(dir(breast_ds))
print(breast_ds['DESCR'])

['DESCR', 'data', 'data_module', 'feature_names', 'filename', 'frame', 'target', 'target_names']
.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features we

#### Prepare the training and test set

In [3]:
from sklearn.model_selection import train_test_split

X = breast_ds.data
y = 1 - breast_ds.target # cancer 1; non-cancer 0
print('size of the feature vectors:', X.shape)
print('size of y:', y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print('size of the training set:', y_train.shape)
print('size of the test set:', y_test.shape)

size of the feature vectors: (569, 30)
size of y: (569,)
size of the training set: (455,)
size of the test set: (114,)


#### Create a NN model using Keras Sequential model
- Design a NN
    1. Input layer
        - how do we determine the number of input nodes?
        - we do not need to define it explicitly

    2. Hidden layers
        - the number of hidden layers
        - the number of hidden nodes/units for each layer
        - activation function

    3. Output layer
        - how many output nodes do we need for this task?
        - activation function

    4. Choose the optimizer(optimization function) and loss/cost function
        - optimizer: Stachastic Gradient Descent (SGD)
        - cost function: Cross-entropy
        
        
 - Create a NN in two different ways

In [4]:
# 1) create a NN by adding layers to an empty sequential model
model1 = keras.models.Sequential() # an empty model
model1.add(keras.layers.Dense(units = 5, # the number of nodes
                              input_dim=30,
                              activation = 'relu')) #hidden layer 1

model1.add(keras.layers.Dense(units = 5,     
                              input_dim=5,
                              activation = 'relu'))# hidden layer 2

model1.add(keras.layers.Dense(units = 2,     
                              activation='softmax')) #output  layer

sgd_optimizer = keras.optimizers.SGD(learning_rate=0.00001) # optimizer
model1.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 155       
                                                                 
 dense_1 (Dense)             (None, 5)                 30        
                                                                 
 dense_2 (Dense)             (None, 2)                 12        
                                                                 
Total params: 197
Trainable params: 197
Non-trainable params: 0
_________________________________________________________________


2023-03-29 15:32:27.046239: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 15:32:27.047743: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
# 2) create a NN using a list of layers
model1 = keras.models.Sequential(
        [
            keras.layers.InputLayer(input_shape=(30, )), # input layer
            keras.layers.Dense(units = 500, activation = 'sigmoid'), # hidden layer
            keras.layers.Dense(units=2, activation='softmax') # ouput layer
        ] # list of layers
)
sgd_optimizer = keras.optimizers.SGD(learning_rate =0.0001)
model1.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 500)               15500     
                                                                 
 dense_4 (Dense)             (None, 2)                 1002      
                                                                 
Total params: 16,502
Trainable params: 16,502
Non-trainable params: 0
_________________________________________________________________


#### Train the model
- Prepare the onehot vectors for the training targets. Onehot-vector encoding using keras.utils.to_categorical() function
![onehot](onehot.png)
- Train the model by using the fit() function
    - input arguments of the fit function:
        - batch_size: 4, 8, 16, 32, 64,.... Training set is splitted into small batches, and one batch will be feeded into the model one time. 
        - epochs (the number of training rounds using the whole training set): 100, 200, 300, ...
        - validation_split: 0.2. Validation set is applied to check model performance and avoid overfitting
        - verbose: 0, 1. Output intermediate training results
        
    - return argument:
        - h.history

In [9]:
# prepare one-hot vectors
y_train_onehot = keras.utils.to_categorical(y_train)

print(X_train.shape)

# train the model
history = model1.fit(X_train, y_train_onehot, 
           batch_size = 16,
           epochs = 100,
           verbose = 2)

#After class practice: train the model using different arguments

TypeError: 'tuple' object is not callable

In [7]:
#  Evaluation 
import numpy as np

y_train_pred = np.argmax(model1.predict(X_train), axis=-1)
train_acc = sum(y_train == y_train_pred)/y_train.shape[0]

y_test_pred = np.argmax(model1.predict(X_test), axis=-1)
test_acc = sum(y_test == y_test_pred)/y_test.shape[0]

print('training acc is', train_acc)
print('test acc is', test_acc)

training acc is 0.8901098901098901
test acc is 0.9298245614035088


In [8]:
#after the class, please calculate the recall rate of the trained NN

